In [1]:
import glob
import torch
import torchvision
import numpy as np
import sys
import cv2
import argparse
from PIL import Image
import pickle
from torchvision.transforms import transforms as transforms
import os

In [2]:
# initialize the model
model = torchvision.models.detection.keypointrcnn_resnet50_fpn(
    pretrained=True, num_keypoints=17
)
# set the computation device
device = torch.device(0)

# load the model on to the computation device and set to eval mode
model.to(device).eval()

# transform to convert the image to tensor
transform = transforms.Compose([
    transforms.ToTensor()
])

In [ ]:
image_files = sorted(glob.glob('../data/raw/*.jpg'))

out_dir = '../data/figures/'
try:
    os.mkdir(out_dir)
except:
    pass

for i, fname in enumerate(image_files):
    sys.stdout.write('\r\t%d\t' % i)
    image = Image.open(fname).convert('RGB')

    # NumPy copy of the image for OpenCV functions
    img = np.array(image, dtype=np.float32) / 255

    # transform the image
    image = transform(image)

    # add a batch dimension
    image = image.unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)
    
    
    H, W, _ = img.shape
    
    boxes = output[0]['boxes'].cpu()
    scores = output[0]['scores'].detach().cpu().numpy()
    for j, b in enumerate(boxes):
        if scores[j] < 0.9:
            continue
        
        x0, y0, x1, y1 = b
        w = x1 - x0
        h = y1 - y0
        
        i0 = max(int(x0 - w/4), 0)
        i1 = min(int(x1 + w/4), W)
        
        j0 = max(int(y0 - h/4), 0)
        j1 = min(int(y1 + h/4), H)
        
        im = img[j0:j1, i0:i1]
        
        outfile = out_dir + fname.split('/')[-1].rstrip('.jpg') + '_%03d' % j + '.pkl'
        
        with open(outfile, 'wb') as fout:
            pickle.dump(im, fout)

	0	

/home/alan/miniconda3/envs/mytorch/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272204863/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


	8	

/home/alan/miniconda3/envs/mytorch/lib/python3.9/site-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


	58	